In [ ]:
from ecmwfapi import ECMWFDataServer

# To run this example, you need an API key 
# available from https://api.ecmwf.int/v1/key/

server = ECMWFDataServer()

server.retrieve({
    "class": "ei",
    "dataset": "interim",
    "date": "2017-01-01/to/2017-12-31",
    "expver": "1",
    "grid": "0.75/0.75",
    "levtype": "sfc",
    "param": "167.128", # 2 metre temperature
    "step": "0",
    "stream": "oper",
    "time": "00:00:00/06:00:00/12:00:00/18:00:00",
    "type": "an",
    "format": "netcdf",
    "target": "temperature_ei_an_2017.nc",
})

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeat

In [2]:
ds = xr.open_mfdataset("temperature_ei_an_2017.nc")

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.Mollweide())
ax.stock_img()
plt.show()

Geometry must be a Point or LineString
Geometry must be a Point or LineString


In [ ]:
def make_figure():
    fig = plt.figure(figsize=(8, 3))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # generate a basemap with country borders, oceans and coastlines
    ax.add_feature(cfeat.LAND)
    ax.add_feature(cfeat.OCEAN)
    ax.add_feature(cfeat.COASTLINE)
    ax.add_feature(cfeat.BORDERS, linestyle='dotted')
    return fig, ax

make_figure();

In [ ]:
area = ds.t2m.sel(longitude=slice(270, 310), latitude=slice(30, 4))

In [ ]:
_, ax = make_figure()
# plot the temperature field
grid = area.sel(time='2017-01-01T12:00:00')
grid.plot(ax=ax, transform=ccrs.PlateCarree());

In [ ]:
import matplotlib.animation as animation

fig, ax = make_figure()

frames = area.time.size        # Number of frames
min_value = area.values.min()  # Lowest value
max_value = area.values.max()  # Highest value


def draw(frame, add_colorbar):
    grid = area[frame]
    contour = grid.plot(ax=ax, transform=ccrs.PlateCarree(),
                        add_colorbar=add_colorbar, vmin=min_value, vmax=max_value)
    title = u"%s — %s" % (ds.t2m.long_name, str(area.time[frame].values)[:19])
    ax.set_title(title)
    return contour


def init():
    return draw(0, add_colorbar=True)


def animate(frame):
    return draw(frame, add_colorbar=False)


ani = animation.FuncAnimation(fig, animate, frames, interval=0.01, blit=False,
                              init_func=init, repeat=False)
ani.save('images/temperature_ei_an_2017.mp4', writer=animation.FFMpegWriter(fps=8))
plt.close(fig)

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

plt.show()